### Backtracking

Backtracking ist ein systematisches Ausprobieren. Wenn wir in eine Sackgasse geraten sind, gehen wir zurück (backtrack) und probieren die nächste Variante aus. Meistens sind alle möglichen Lösungen gefragt.

Backtracking ist anwendbar, wenn wir eine Lösung schrittweise aufbauen können und wenn wir für eine
Teillösung die möglichen Kandidaten aufzählen können, die die Teillösung erweitern. 

Eine Teillösung bezeichnen wir mit $x$. $x$ ist eine Liste mit Entscheidungen $x[0]...x[k]$, die zur Lösung führen sollen.



In [ ]:
def back(x):
    global solutions
    if isSolution(x):
        solutions.append(x.copy())
    else:
        for cand in candidates(x):
            if isGood(cand,x):
                x.append(cand)
                back(x)
                x.pop()
                
def isSolution(x):
    '''
    returns: True, wenn der Lösungsvektor x eine Lösung darstellt
    '''
    pass

def candidates(x):
    '''
    returns: Liste von Entscheidungen, die den Lösungsvektor x um eine Stufe erweitern
    '''
    pass

def isGood(cand, x):
    '''
    returns: True, wenn die Entscheidung cand den Lösungsvektor x sinnvoll erweitert.
    '''
               
solutions = []
back([])

#### Beispiel: Alle Kombinationen 3 aus 5

Aus den Zahlen von 1-5 sollen alle möglichen 3er Kombinationen ermittelt werden.

In [4]:
# Das lässt sich  natürlich mit itertools lösen
import itertools as it
for x in it.combinations(range(1,6),r=3):
    print(x)

(1, 2, 3)
(1, 2, 4)
(1, 2, 5)
(1, 3, 4)
(1, 3, 5)
(1, 4, 5)
(2, 3, 4)
(2, 3, 5)
(2, 4, 5)
(3, 4, 5)


In [2]:
# Das lässt sich auch mit backtracking losen
'''
x ist ein Lösungsvektor, d.h. eine Liste von Entscheidungen, die getroffen wurden auf der Suche
nach einer Lösung.
'''
def back(x):
    if isSolution(x):
        solutions.append(x.copy())
    else:
        for cand in candidates(x):
            if isGood(cand,x):
                x.append(cand)
                back(x)
                x.pop()
                
def isSolution(x):
    '''
    returns: True, wenn der Lösungsvektor x eine Lösung darstellt
    '''
    return len(x) == 3  

def candidates(x):
    '''
    returns: Liste von Entscheidungen, die den Lösungsvektor x um eine Stufe erweitern
    '''
    return list(range(1,6))


def isGood(cand, x):
    '''
    returns: True, wenn die Entscheidung cand den Lösungsvektor x sinnvoll erweitert.
    '''
    return cand not in x and (len(x) == 0 or cand > x[-1])  # Lösungen sind aufsteigend sortiert   
    

               
solutions = []
back([])
solutions

[[1, 2, 3],
 [1, 2, 4],
 [1, 2, 5],
 [1, 3, 4],
 [1, 3, 5],
 [1, 4, 5],
 [2, 3, 4],
 [2, 3, 5],
 [2, 4, 5],
 [3, 4, 5]]

Die folgende Version fasst die einzelnen Elemente zusammen und nennt die Variablen anders. Der Lösungsvektor heißt jetzt *path* und *x* ist ein Element des Lösungsvektors.

In [21]:
def back(path):
    if len(path) == 3:
        solutions.append(path.copy())
    else:
        for x in range(1,6):
            if x not in path and (len(path) == 0 or x > path[-1]):
                path.append(x)
                back(path)
                path.pop()
                
solutions = []
back([])
print(solutions)

[[1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5]]


Im Vergleich dazu die rekursive Tiefensuche. Die nextstates werden als Liste erzeugt. Beim backtracking wird derselbe Pfad immer wieder verändert. Erst wenn eine Lösung gefunden wird, wird eine Kopie gesichert.

In [23]:
def nextstates(path):
    if len(path) == 3: return []
    tmp = []
    for x in range(1, 6):
        if x not in path and (len(path) == 0 or x > path[-1]):
            tmp.append(path+[x])
    return tmp

def dfs(path):
    if len(path) == 3:
        solutions.append(path.copy())
    else:
        for p in nextstates(path):
            dfs(p)
                
solutions = []
dfs([])
print(solutions) 

[[1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5]]


### Breitensuche

Wir beginnen mit einem startstate. Mögliche Folgeaktionen führen uns zu den nextstates. Die Knoten in unserem Suchbaum sind nicht die Aktionen, sondern die aus den Aktionen resultierenden Zustände. Unterschiedliche Aktionsfolgen können dieselben Zustände erzeugen. Dieselben Zustände wollen wir nicht nicht mehrfach untersuchen.

Die states modellieren wir als Tuple, weil wir sie als keys in dem dictionary prev verwenden wollen. Das dictionary prev speichert für ein Tuple den Vorgänger. Es dient zwei Zwecken: 1. Wir können feststellen, ob wir ein state im Laufe der Suche schon einmal gesehen haben. 2. Wir können die Abfolge der Aktionen rekonstruieren, die uns vom start zum ziel führen.


In [72]:
from collections import deque
def bfs(s):
    ''' 
    s = startstate als Tuple
    returns:
        prev, goalstate falls Suche erfolgreich
        None, None falls Suche nicht erfolgreich
        
        prev: dict mit den Vorgängern der states
        goalstate: state, der den goaltest besteht
    '''
    frontier =  deque([s])
    prev = {s:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    '''
    returns: Pfad vom startstate zum goalstate
    '''
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def goaltest(state):
    '''
    state: Spielstellung
    returns: True, wenn state den goaltest besteht
    '''  
    pass

def nextstates(state):
    '''
    returns: Liste mit Tuples, die Liste mit den möglichen Folgestates von state
    '''
    pass
 

#### Beispiel: 8-Puzzle

In [73]:
from collections import deque
def bfs(s):
    frontier =  deque([s])
    prev = {s:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    '''
    returns: Pfad vom startstate zum goalstate
    '''
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def goaltest(state):
    '''
    state: Spielstellung
    returns: True, wenn state den goaltest besteht
    '''  
    return state == (0,1,2,3,4,5,6,7,8)

# wir vereinbaren die Reihenfolge: oben-unten-links-rechts
tausch = {0:[3,1], 1:[4,0,2],2:[5,1],3:[0,6,4],4:[1,7,3,5],5:[2,8,4],6:[3,7],7:[4,6,8],8:[5,7]}
def nextstates(state):
    tmp = []
    i = state.index(0)
    for j in tausch[i]:
        a = list(state)
        a[i],a[j] = a[j],a[i]
        tmp.append(tuple(a))
    return tmp

def reconstructActions(path):
    tmp = []
    for i in range(len(path)-1):
        s1 = path[i]
        s2 = path[i+1]
        i1 = s1.index(0)
        i2 = s2.index(0)
        if i2 == i1+1:   # right
            tmp.append('R')
        elif i2 == i1-1: # left
            tmp.append('L')
        elif i2 == i1-3: # up
            tmp.append('U')
        elif i2 == i1+3: # donw
            tmp.append('D')
        else:
            raise RuntimeError('Fehler')
    return tmp
    

In [74]:
%%time
startstate = (7,2,4,5,0,6,8,3,1)
prev, goalstate = bfs(startstate)
path = reconstructPath(prev, goalstate)
actions = reconstructActions(path)
print(*actions)
print('Anzahl Züge:',len(actions))

L U R D D L U R R U L L D R R D L U R U L D D L U U
Anzahl Züge: 26
CPU times: total: 266 ms
Wall time: 268 ms


### Tiefensuche

Der Code für die iterative Tiefensuche gleicht dem der Breitensuche, nur verwenden wir als frontier einen Stack statt einer Schlange.

In [77]:
def dfs(s):
    frontier =  [s]
    prev = {s:None}
    while frontier:
        state = frontier.pop()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

#### Beispiel: Erreichbarkeit in einem Graphen

Wir wollen in dem Graphen alle erreichbaren Knoten durchlaufen, also entfällt der goaltest. Wenn uns der Weg zu den Knoten nicht interessiert, ersetzen wir das dict *prev* durch das set *visited*. Die nextstates sind die Nachbarn des Knotens.

<img src='graph1.png' width="300">

In [4]:
G = {'a':set('bc'), 'b':set('d'), 'c':set('bd'), 'd':set('b')}

def dfs(s):
    frontier =  [s]
    visited = set()
    while frontier:
        w = frontier.pop()  
        visited.add(w)
        for v in G[w]:
            if v not in visited:
                frontier.append(v)
    return visited

print(dfs('a'))
print(dfs('c'))

{'a', 'd', 'c', 'b'}
{'c', 'd', 'b'}


Die rekursive Variante

In [9]:
def dfs(v):  
    visited[v] = True
    for w in G[v]:
        if not visited[w]:
            dfs(w) 

visited =  {v : False for v in G}      
dfs('c')
print([v for v in G if visited[v]])


['b', 'c', 'd']
